In [2]:
!pip install librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 34.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.8/952.8 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 24.2 MB/s eta 0:00:0000:0100:01
  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23702 sha256=bf49b2d0d09e5e1fa1bf739ca18bbd4cc42e0f1910994fdffa38f7164663f90e
  Stored in directory: /root/.cache/pip/wheels/e4/76/a4/cfb55573167a1f5bde7d7a348e95e509c64b2c3e8f921932c3
Successfully built audioread
  Attempting uninstall: setuptools
    Found existing installation: setupto

In [1]:
########################################################################
# import default libraries
########################################################################
import os
import sys
import gc
########################################################################


########################################################################
# import additional libraries
########################################################################
import numpy as np
import scipy.stats
# from import
from tqdm import tqdm
try:
    from sklearn.externals import joblib
except:
    import joblib
# original lib
import common as com
import keras_model
import pandas as pd
import keras
import librosa
import tensorflow as tf
import librosa.display
from generator import SpecAugment
############

In [2]:
############################################################


########################################################################
# load parameter.yaml
########################################################################
param = com.yaml_load()
########################################################################
saved_weight = os.path.join(param["P_MODELSAVE"], 'dataweights.{epoch:02d}-{val_acc:.2f}.hdf5')

modelchk = keras.callbacks.ModelCheckpoint(saved_weight, 
                                      monitor='val_acc', 
                                      verbose=1,
                                      save_best_only=True, 
                                      save_weights_only=False,
                                      mode='auto',
                                      period=2)

tensorboard = keras.callbacks.TensorBoard(log_dir=param["P_LOGS"],
                                          histogram_freq=0,
                                          write_graph=True,
                                          write_images=True)

csv_logger = keras.callbacks.CSVLogger(f'{param["P_LOGS"]}/keras_log.csv',
                                       append=True)

# model_unet = keras_model.get_model(input_shape=(1,param["feature"]["n_mels"],param["feature"]["n_frames"]), lr = param["fit"]["lr"])

########################################################################
# visualizer
########################################################################
class visualizer(object):
    def __init__(self):
        import matplotlib.pyplot as plt
        self.plt = plt
        self.fig = self.plt.figure(figsize=(7, 5))
        self.plt.subplots_adjust(wspace=0.3, hspace=0.3)

    def loss_plot(self, loss, val_loss):
        """
        Plot loss curve.

        loss : list [ float ]
            training loss time series.
        val_loss : list [ float ]
            validation loss time series.

        return   : None
        """
        ax = self.fig.add_subplot(1, 1, 1)
        ax.cla()
        ax.plot(loss)
        ax.plot(val_loss)
        ax.set_title("Model loss")
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        ax.legend(["Train", "Validation"], loc="upper right")

    def save_figure(self, name):
        """
        Save figure.

        name : str
            save png file path.

        return : None
        """
        self.plt.savefig(name)


########################################################################


########################################################################
# get data from the list for file paths
########################################################################
def file_list_to_data(file_list,
                      msg="calc...",
                      n_mels=64,
                      n_frames=5,
                      n_hop_frames=1,
                      n_fft=1024,
                      hop_length=512,
                      power=2.0):
    """
    convert the file_list to a vector array.
    file_to_vector_array() is iterated, and the output vector array is concatenated.

    file_list : list [ str ]
        .wav filename list of dataset
    msg : str ( default = "calc..." )
        description for tqdm.
        this parameter will be input into "desc" param at tqdm.

    return : numpy.array( numpy.array( float ) )
        data for training (this function is not used for test.)
        * dataset.shape = (number of feature vectors, dimensions of feature vectors)
    """

    # iterate file_to_vector_array()
    for idx in tqdm(range(len(file_list)), desc=msg):
        vectors = com.file_to_vectors(file_list[idx],
                                                n_mels=n_mels,
                                                n_frames=n_frames,
                                                n_fft=n_fft,
                                                hop_length=hop_length,
                                                power=power)
        # vectors_masked = com.spec_augment(vectors)
        if idx == 0:
            data = np.zeros((len(file_list), vectors.shape[0], vectors.shape[1]), float)
            # data_masked = np.zeros((len(file_list), vectors_masked.shape[1], vectors_masked.shape[0]), float)
        data[idx, :, :] = vectors
        data[idx,:,:] = librosa.power_to_db(data[idx,:,:])
        # data_masked[idx, :, :] = vectors_masked.T
    return data


In [3]:
mode = True

# make output directory
os.makedirs(param["model_directory"], exist_ok=True)

# initialize the visualizer
visualizer = visualizer()

# load base_directory list
dirs = com.select_dirs(param=param, mode=mode)
idx = 0
# target_dir = "/content/drive/MyDrive/dev_data/car8"
target_dir = "dev_data/car3"
# loop of the base directory
print("\n===========================")
print("[{idx}/{total}] {target_dir}".format(target_dir=target_dir, idx=idx+1, total=len(dirs)))

# set path
machine_type = os.path.split(target_dir)[1]
model_file_path = "{model}/model_{machine_type}.hdf5".format(model=param["model_directory"],
                                                                machine_type=machine_type)

history_img = "{model}/history_{machine_type}.png".format(model=param["model_directory"],
                                                            machine_type=machine_type)
# pickle file for storing anomaly score distribution
score_distr_file_path = "{model}/score_distr_{machine_type}.pkl".format(model=param["model_directory"],
                                                                        machine_type=machine_type)




2022-10-11 10:09:54,775 - INFO - load_directory <- development



[1/8] dev_data/car3


<Figure size 504x360 with 0 Axes>

In [4]:
# generate dataset
print("============== DATASET_GENERATOR ==============")

# get file list for all sections
# all values of y_true are zero in training
files, y_true = com.file_list_generator(target_dir=target_dir,
                                        section_name="*",
                                        dir_name="train",
                                        mode=mode)



data = file_list_to_data(files,
                            msg="generate train_dataset",
                            n_mels=param["feature"]["n_mels"],
                            n_frames=1,
                            n_hop_frames=param["feature"]["n_mels"],
                            n_fft=param["feature"]["n_fft"],
                            hop_length=param["feature"]["hop_length"],
                            power=param["feature"]["power"])
data = np.concatenate((data, data), axis=0)
j = param["feature"]["n_mels"]
y = param["feature"]["n_frames"]
n = machine_type
print(data.shape)
# data_masked, data = masking(data)
# data = data.reshape(1222, data.shape[1], data.shape[2], 1)
# data_masked =  data_masked.reshape(1222, data.shape[1], data.shape[2], 1)



2022-10-11 10:09:54,789 - INFO - target_dir : dev_data/car3_*
2022-10-11 10:09:54,797 - INFO - #files : 611


============== DATASET_GENERATOR ==============



generate train_dataset: 100%|██████████| 611/611 [00:12<00:00, 47.40it/s]


(1222, 128, 512)


In [5]:
# j = param["feature"]["n_mels"]
# y = param["feature"]["n_frames"]
# data = np.load(f"data__{j}_{y}.npy")

In [6]:
from sklearn.model_selection import train_test_split


In [7]:
# number of vectors for each wave file
# train model
print("============== MODEL TRAINING ==============")
model = keras_model.get_model_3((param["feature"]["n_frames"],param["feature"]["n_mels"], 1))
model_opt = tf.keras.optimizers.Adam(learning_rate=4)

model.compile(optimizer=model_opt, loss='mse', metrics=['accuracy'])

model.summary()

# history = model.fit(x=data_masked,
#                     y=data,
#                     epochs=200,
#                     batch_size=32,
#                     shuffle=param["fit"]["shuffle"],
#                     validation_split=0.1,
#                     verbose=param["fit"]["verbose"])

history = model.fit(SpecAugment(data, 32),
                 epochs=200,
                 shuffle=param["fit"]["shuffle"],
                 verbose=param["fit"]["verbose"],
                 workers = 6)



pd.DataFrame.from_dict(model.history.history).to_csv(f'history_{machine_type}.csv',index=False)


# # calculate y_pred for fitting anomaly score distribution
# y_pred = []
# start_idx = 0
# for file_idx in range(len(files)):
#         y_pred.append(np.mean(np.square(data[file_idx,: ,  :] 
#                                 - model.predict(data[file_idx,: , :]))))
#         start_idx += n_vectors_ea_file

# # fit anomaly score distribution
# shape_hat, loc_hat, scale_hat = scipy.stats.gamma.fit(y_pred)
# gamma_params = [shape_hat, loc_hat, scale_hat]
# joblib.dump(gamma_params, score_distr_file_path)

model.save(model_file_path)
com.logger.info("save_model -> {}".format(model_file_path))
visualizer.loss_plot(history.history["loss"], history.history["val_loss"])
visualizer.save_figure(history_img)

print("============== END TRAINING ==============")

del data
del model
gc.collect()


============== MODEL TRAINING ==============


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 32, 8, 128), (None, 64, 16, 128)]